# Introduction to Random Forests

In [1]:
#Auto-update modules
%load_ext autoreload
%autoreload 2

#displays plots inside the notebook
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

# Overview

For this competition, you are predicting the sale price of bulldozers sold at auctions. The data for this competition is split into three parts:

    Train.csv is the training set, which contains data through the end of 2011.
    Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012. You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
    Test.csv is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition.

The key fields are in train.csv are:

    SalesID: the unique identifier of the sale
    MachineID: the unique identifier of a machine. A machine can be sold multiple times
    saleprice: what the machine sold for at auction (only provided in train.csv)
    saledate: the date of the sale


The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuaration.  The data is sourced from auction result postings and includes information on usage and equipment configurations.

Fast Iron is creating a "blue book for bull dozers," for customers to value what their heavy equipment fleet is worth at auction.

In [ ]:
#importing datasets
train = pd.read_csv('data/train.csv', low_memory=False, parse_dates=["saledate"])
train.head()

In [ ]:
train.T

In [ ]:
pip install -q fastai==0.6

In [ ]:
test = pd.read_csv('https://storage.googleapis.com/kaggle-competitions-data/kaggle/3316/Test.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1563917599&Signature=Qp2ny2LTLaJf3OjyzsGE6JkxiT5dbG%2FXBQY5gGceG4RLSmD4LW1xQ2DuDjR0i5cPu1iaK37AQbhAihh8lJl%2BcvIQ2Bs67v%2FAOb%2FGQ3QNBSP2uWOTT4LF4Z8kx%2BmbuEKOyEsk5AWQoLOOAvCuEUM1kZVm%2B6vhhYlUmJ2tjrlM2CtNLH03%2FUTcd%2Bl7xqyT4WWbqWIGyV%2BmbK6LrGG9fj4Vg6IiCWxkejlBGCbqvTFqqVbNnpMCzzV2zxwoEKFtolNXCtpDOaJxC0wmy9JSF4BD5tscci%2FOzo3TdNstAS2enfj%2BULFV5rk%2F0Ectivy8x0nEdhrgeH%2FNcQIelDJD7sLp%2BA%3D%3D')
test.head()

In [ ]:
train.info().head()

In [ ]:
def display_all(train):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(train)
        
train.head()

In [ ]:
train.SalePrice = np.log(train.SalePrice)
train.SalePrice.head()

In [ ]:
# np.log(train.MachineHoursCurrentMeter)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestRegressor(n_jobs=-1)
# rf.fit(train.drop('SalePrice', axis=1), train.SalePrice)

#### This dataset contains a mix of continuous and categorical variables.

The following method extracts particular date fields from a complete datetime for the purpose of constructing categoricals. You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.


In [ ]:
fld = train.saledate
fld.head(4)

In [ ]:
fld.dt.year.head(4)

In [ ]:
add_datepart(train, 'saledate')
train.saleYear.head()

In [ ]:
train_cats(train)

In [ ]:
train.UsageBand.cat.categories

In [ ]:
train.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)
#Normally, pandas will continue displaying the text categories, while treating them as numerical data internally. 
# Optionally, we can replace the text categories with numbers, which will make this variable non-categorical, like so:.
train.UsageBand = train.UsageBand.cat.codes

In [ ]:
#We're still not quite done - for instance we have lots of missing values, which we can't pass directly to a random forest.
display_all(train.isnull().sum().sort_index()/len(train))

In [ ]:
df, y, nas = proc_df(train, 'SalePrice')

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)